In [60]:
import sqlite3
import json

def load_json_objects(file_path):
    data = []
    seen_ids = set()  # Track unique IDs
    duplicates = 0    # Count duplicates
    
    with open(file_path) as f:
        for line in f:
            if line.strip():
                record = json.loads(line)
                # Get the unique ID from the record (using _id.$oid)
                record_id = record.get('_id', {}).get('$oid')
                if record_id not in seen_ids:
                    seen_ids.add(record_id)
                    data.append(record)
                else:
                    duplicates += 1
                    
    print(f"Loaded {len(data)} unique records from {file_path}")
    print(f"Found {duplicates} duplicate records")
    return data

# Create an in-memory SQLite database for demo purpose, faster and easy to run
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create USER table
cursor.execute("""
CREATE TABLE USER (
    user_id TEXT PRIMARY KEY,
    state TEXT,
    created_date INTEGER,
    last_login INTEGER,
    role TEXT,
    active BOOLEAN,
    sign_up_source TEXT
);
""")

# Create BRAND table
cursor.execute("""
CREATE TABLE BRAND (
    brand_id TEXT PRIMARY KEY,
    barcode TEXT,
    name TEXT,
    brand_code TEXT,
    category TEXT,
    category_code TEXT,
    cpg_id TEXT,
    top_brand BOOLEAN
);
""")

# Create RECEIPT table
cursor.execute("""
CREATE TABLE RECEIPT (
    receipt_id TEXT PRIMARY KEY,
    user_id TEXT,
    bonus_points_earned INTEGER,
    bonus_points_earned_reason TEXT,
    create_date INTEGER,
    date_scanned INTEGER,
    finished_date INTEGER,
    modify_date INTEGER,
    points_awarded_date INTEGER,
    points_earned DECIMAL,
    purchase_date INTEGER,
    purchased_item_count INTEGER,
    total_spent DECIMAL,
    rewards_receipt_status TEXT,
    FOREIGN KEY (user_id) REFERENCES USER(user_id)
);
""")

# Create RECEIPT_ITEM table
cursor.execute("""
CREATE TABLE RECEIPT_ITEM (
    receipt_item_id TEXT PRIMARY KEY,
    receipt_id TEXT,
    barcode TEXT,
    description TEXT,
    final_price DECIMAL,
    item_price DECIMAL,
    quantity_purchased INTEGER,
    partner_item_id TEXT,
    needs_fetch_review BOOLEAN,
    prevent_target_gap_points BOOLEAN,
    user_flagged_barcode TEXT,
    user_flagged_new_item BOOLEAN,
    user_flagged_price DECIMAL,
    user_flagged_quantity INTEGER,
    brand_code TEXT,
    FOREIGN KEY (receipt_id) REFERENCES RECEIPT(receipt_id)
);
""")

# Load JSON data
users_data = load_json_objects('users.json')
brands_data = load_json_objects('brands.json')
receipts_data = load_json_objects('receipts.json')

# Insert USERS data
for user in users_data:
    try:
        cursor.execute("""
            INSERT INTO USER (user_id, state, created_date, last_login, role, active, sign_up_source)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (
            user['_id']['$oid'],
            user.get('state', ''),
            user.get('createdDate', {}).get('$date', 0),
            user.get('lastLogin', {}).get('$date', 0),
            user.get('role', ''),
            user.get('active', False),
            user.get('signUpSource', '')
        ))
    except sqlite3.IntegrityError:
        print(f"Skipping duplicate user_id: {user['_id']['$oid']}")
        continue

# Insert BRANDS data
for brand in brands_data:
    try:
        cursor.execute("""
            INSERT INTO BRAND (brand_id, barcode, brand_code, category, category_code, cpg_id, top_brand, name)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            brand['_id']['$oid'],
            brand.get('barcode', ''),
            brand.get('brandCode', ''),
            brand.get('category', ''),
            brand.get('categoryCode', ''),
            brand.get('cpg', {}).get('$id', {}).get('$oid', ''),
            brand.get('topBrand', False),
            brand.get('name', '')
        ))
    except sqlite3.IntegrityError:
        print(f"Skipping duplicate brand_id: {brand['_id']['$oid']}")
        continue

# Insert RECEIPTS data & process nested receipt items
for receipt in receipts_data:
    try:
        receipt_id = receipt['_id']['$oid']
        cursor.execute("""
            INSERT INTO RECEIPT (
                receipt_id,
                user_id,
                bonus_points_earned,
                bonus_points_earned_reason,
                create_date,
                date_scanned,
                finished_date,
                modify_date,
                points_awarded_date,
                points_earned,
                purchase_date,
                purchased_item_count,
                total_spent,
                rewards_receipt_status
            )
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            receipt_id,
            receipt.get('userId', ''),
            receipt.get('bonusPointsEarned', 0),
            receipt.get('bonusPointsEarnedReason', ''),
            receipt.get('createDate', {}).get('$date', 0),
            receipt.get('dateScanned', {}).get('$date', 0),
            receipt.get('finishedDate', {}).get('$date', 0),  # If not provided, defaults to 0
            receipt.get('modifyDate', {}).get('$date', 0),
            receipt.get('pointsAwardedDate', {}).get('$date', 0),  # If not provided, defaults to 0
            receipt.get('pointsEarned', 0),
            receipt.get('purchaseDate', {}).get('$date', 0),
            receipt.get('purchasedItemCount', 0),
            receipt.get('totalSpent', 0),
            receipt.get('rewardsReceiptStatus', '')
        ))
    except sqlite3.IntegrityError:
        print(f"Skipping duplicate receipt_id: {receipt_id}")
        continue

    # Process nested receipt items using the key "rewardsReceiptItemList"
    receipt_items = receipt.get("rewardsReceiptItemList", [])
    for index, item in enumerate(receipt_items):
        receipt_item_id = f"{receipt_id}-{index}"
        try:
            cursor.execute("""
                INSERT INTO RECEIPT_ITEM (
                    receipt_item_id,
                    receipt_id,
                    barcode,
                    description,
                    final_price,
                    item_price,
                    quantity_purchased,
                    partner_item_id,
                    needs_fetch_review,
                    prevent_target_gap_points,
                    user_flagged_barcode,
                    user_flagged_new_item,
                    user_flagged_price,
                    user_flagged_quantity,
                    brand_code
                )
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, (
                receipt_item_id,
                receipt_id,
                item.get('barcode', ''),
                item.get('description', ''),
                item.get('finalPrice', 0),
                item.get('itemPrice', 0),
                item.get('quantityPurchased', 0),
                item.get('partnerItemId', ''),
                False,  # Default for needs_fetch_review
                False,  # Default for prevent_target_gap_points
                '',     # Default for user_flagged_barcode
                False,  # Default for user_flagged_new_item
                0,      # Default for user_flagged_price
                0,      # Default for user_flagged_quantity
                ''      # Default for brand_code
            ))
        except sqlite3.IntegrityError:
            print(f"Skipping duplicate receipt_item_id: {receipt_item_id}")
            continue

conn.commit()

# Verification: print counts for each table
# Print columns and sample rows for each table
tables = ['USER', 'BRAND', 'RECEIPT', 'RECEIPT_ITEM']

for table in tables:
    print(f"\nTable: {table}")
    
    # Get table schema using PRAGMA table_info
    cursor.execute(f"PRAGMA table_info({table});")
    columns_info = cursor.fetchall()  # Each row is (cid, name, type, notnull, dflt_value, pk)
    columns = [col[1] for col in columns_info]
    print("Columns:", columns)
    
    # Fetch and print sample rows
    cursor.execute(f"SELECT * FROM {table} LIMIT 5")
    rows = cursor.fetchall()
    print("Sample rows:")
    for row in rows:
        print(row)


Loaded 212 unique records from users.json
Found 283 duplicate records
Loaded 1167 unique records from brands.json
Found 0 duplicate records
Loaded 1119 unique records from receipts.json
Found 0 duplicate records

Table: USER
Columns: ['user_id', 'state', 'created_date', 'last_login', 'role', 'active', 'sign_up_source']
Sample rows:
('5ff1e194b6a9d73a3a9f1052', 'WI', 1609687444800, 1609687537858, 'consumer', 1, 'Email')
('5ff1e1eacfcf6c399c274ae6', 'WI', 1609687530554, 1609687530597, 'consumer', 1, 'Email')
('5ff1e1e8cfcf6c399c274ad9', 'WI', 1609687528354, 1609687528392, 'consumer', 1, 'Email')
('5ff1e1b7cfcf6c399c274a5a', 'WI', 1609687479626, 1609687479665, 'consumer', 1, 'Email')
('5ff1e1f1cfcf6c399c274b0b', 'WI', 1609687537564, 1609687537599, 'consumer', 1, 'Email')

Table: BRAND
Columns: ['brand_id', 'barcode', 'name', 'brand_code', 'category', 'category_code', 'cpg_id', 'top_brand']
Sample rows:
('601ac115be37ce2ead437551', '511111019862', 'test brand @1612366101024', '', 'Baking',

In [59]:
# query the receipt_item table to know the user_flagged_quantity, quantity_purchased and status
query = """
SELECT 
    ri.user_flagged_quantity,
    ri.quantity_purchased,
    r.rewards_receipt_status
FROM RECEIPT_ITEM ri
JOIN RECEIPT r ON ri.receipt_id = r.receipt_id
WHERE r.rewards_receipt_status in ('REJECTED')
LIMIT 10;
"""

cursor.execute(query)
results = cursor.fetchall()
print(results)


[(0, 0, 'REJECTED'), (0, 1, 'REJECTED'), (0, 0, 'REJECTED'), (0, 0, 'REJECTED'), (0, 0, 'REJECTED'), (0, 0, 'REJECTED'), (0, 0, 'REJECTED'), (0, 0, 'REJECTED'), (0, 0, 'REJECTED'), (0, 0, 'REJECTED')]


In [73]:
# What are the top 5 brands by receipts scanned for most recent month?

query = """
SELECT 
    b.brand_id,
    b.name,
    COUNT(DISTINCT r.receipt_id) AS receipts_scanned
FROM RECEIPT r
JOIN RECEIPT_ITEM ri 
    ON r.receipt_id = ri.receipt_id
JOIN BRAND b 
    ON ri.brand_code = b.brand_code
WHERE strftime('%Y', datetime(r.date_scanned/1000, 'unixepoch')) = 
      strftime('%Y', datetime((SELECT MAX(date_scanned) FROM RECEIPT)/1000, 'unixepoch'))
  AND strftime('%m', datetime(r.date_scanned/1000, 'unixepoch')) = 
      strftime('%m', datetime((SELECT MAX(date_scanned) FROM RECEIPT)/1000, 'unixepoch'))
GROUP BY b.brand_id, b.name
ORDER BY receipts_scanned DESC
LIMIT 5;
"""

# Execute the query.
cursor.execute(query)
results = cursor.fetchall()

# Print the results.
print("Top 5 brands based on distinct receipts scanned in the latest month:")
for row in results:
    print(row)

Top 5 brands based on distinct receipts scanned in the latest month:
('5332f5ebe4b03c9a25efd0a7', 'Monster', 13)
('5332f5f2e4b03c9a25efd0a9', 'Eggo', 13)
('5332f5f2e4b03c9a25efd0ab', 'Our Family', 13)
('5332f5f3e4b03c9a25efd0ad', 'Gree Giant', 13)
('5332f5f4e4b03c9a25efd0af', 'Frosted Mini-Wheats', 13)


In [89]:
recent_months_count = 1

query = f"""
WITH recent_months AS (
    SELECT DISTINCT strftime('%Y-%m', datetime(date_scanned/1000, 'unixepoch')) AS month
    FROM RECEIPT
    ORDER BY month DESC
    LIMIT {recent_months_count}
)

SELECT 
      b.brand_id,
      b.name,
      b.barcode,
      ri.user_flagged_barcode,
      COUNT(DISTINCT r.receipt_id) AS receipts_scanned
  FROM RECEIPT r
  JOIN RECEIPT_ITEM ri 
      ON r.receipt_id = ri.receipt_id
  JOIN BRAND b 
      ON ri.barcode = b.barcode
  WHERE strftime('%Y-%m', datetime(r.date_scanned/1000, 'unixepoch'))
          IN (SELECT month FROM recent_months)
  GROUP BY b.brand_id, b.name
  ORDER BY receipts_scanned DESC 
  limit 5;
  """

cursor.execute(query)
results = cursor.fetchall()

# Print the results.
print("Top 5 brands based on distinct receipts scanned in the latest month:")
print(results)
for row in results:
    print(row)

Top 5 brands based on distinct receipts scanned in the latest month:
[('585a972de4b03e62d1ce0e96', 'Tostitos', '511111001485', '', 11), ('5a8c36dbe4b0ccf165fac9e9', 'Swanson', '511111204206', '', 11), ('57d9580ee4b0ac389136a2b6', 'Cracker Barrel Cheese', '511111802358', '', 10), ('5a7e0665e4b0aedb3b84afd4', 'Diet Chris Cola', '511111704140', '', 4), ('5a8c344ae4b0ccf165fac9e5', 'Prego', '511111704140', '', 4)]
('585a972de4b03e62d1ce0e96', 'Tostitos', '511111001485', '', 11)
('5a8c36dbe4b0ccf165fac9e9', 'Swanson', '511111204206', '', 11)
('57d9580ee4b0ac389136a2b6', 'Cracker Barrel Cheese', '511111802358', '', 10)
('5a7e0665e4b0aedb3b84afd4', 'Diet Chris Cola', '511111704140', '', 4)
('5a8c344ae4b0ccf165fac9e5', 'Prego', '511111704140', '', 4)


In [72]:
# How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?
# Example
# Brand   Ranking_this_month  Rainking_last_month
# A        1                  4
# B        2                  2
# C        3                  3
# D        4                  1
# E        5                  N/A
recent_months_count = 2

query = f"""
WITH recent_months AS (
    SELECT DISTINCT strftime('%Y-%m', datetime(date_scanned/1000, 'unixepoch')) AS month
    FROM RECEIPT
    ORDER BY month DESC
    LIMIT {recent_months_count}
),
monthly AS (
    SELECT 
        strftime('%Y-%m', datetime(r.date_scanned/1000, 'unixepoch')) AS month,
        b.brand_id,
        b.name,
        COUNT(DISTINCT r.receipt_id) AS receipts_scanned
    FROM RECEIPT r
    JOIN RECEIPT_ITEM ri ON r.receipt_id = ri.receipt_id
    JOIN BRAND b ON ri.brand_code = b.brand_code
    WHERE strftime('%Y-%m', datetime(r.date_scanned/1000, 'unixepoch'))
          IN (SELECT month FROM recent_months)
    GROUP BY month, b.brand_id, b.name
),
ranked AS (
    SELECT 
        month,
        brand_id,
        name,
        receipts_scanned,
        ROW_NUMBER() OVER (PARTITION BY month ORDER BY receipts_scanned DESC) AS rn
    FROM monthly
)
Select * from ranked

"""

cursor.execute(query)
results = cursor.fetchall()
print(results)

print("Brand | Ranking_this_month | Ranking_last_month")
print("------------------------------------------------")
for row in results:
    print(f"{row[0]} | {row[1]} | {row[2]}")

[('2021-02', '5332f5ebe4b03c9a25efd0a7', 'Monster', 119, 1), ('2021-02', '5332f5f2e4b03c9a25efd0a9', 'Eggo', 119, 2), ('2021-02', '5332f5f2e4b03c9a25efd0ab', 'Our Family', 119, 3), ('2021-02', '5332f5f3e4b03c9a25efd0ad', 'Gree Giant', 119, 4), ('2021-02', '5332f5f4e4b03c9a25efd0af', 'Frosted Mini-Wheats', 119, 5), ('2021-02', '5332f5f5e4b03c9a25efd0b0', 'Betty Crocker', 119, 6), ('2021-02', '5332f5f5e4b03c9a25efd0b1', 'Minute Maid', 119, 7), ('2021-02', '5332f5f6e4b03c9a25efd0b2', 'Coca-Cola', 119, 8), ('2021-02', '5332f5fee4b03c9a25efd0bd', 'Bottled Starbucks', 119, 9), ('2021-02', '5332f603e4b03c9a25efd0bf', 'Lipton', 119, 10), ('2021-02', '5332f605e4b03c9a25efd0c0', 'Amp', 119, 11), ('2021-02', '5332f608e4b03c9a25efd0c1', 'Sierra Mist', 119, 12), ('2021-02', '5332f612e4b03c9a25efd0c4', 'Kickstart', 119, 13), ('2021-02', '5332f6ede4b03c9a25efd0e5', 'Blue Bunny', 119, 14), ('2021-02', '5332f6f4e4b03c9a25efd0ed', 'Peace Tea', 119, 15), ('2021-02', '5332f70ae4b03c9a25efd0f4', 'Barmen', 

In [48]:
# When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
# Assume Finished means the receipt is processed and accepted

query_avg_spend = """
SELECT
    rewards_receipt_status,
    AVG(total_spent) AS avg_spend
FROM RECEIPT
where rewards_receipt_status in ('FINISHED', 'REJECTED')
group by rewards_receipt_status
"""

# Execute the query.
cursor.execute(query_avg_spend)
results = cursor.fetchall()

# Print the results.
print("Rewards Receipt Status | Average Spend")
print("-----------------------------------------")
print(results)


Rewards Receipt Status | Average Spend
-----------------------------------------
[('FINISHED', 80.85430501930502), ('REJECTED', 23.32605633802817)]


# The average spend of finished receipts is greater than the rejected receipts

```py
Rewards Receipt Status | Average Spend
-----------------------------------------
[('FINISHED', 80.85430501930502), ('REJECTED', 23.32605633802817)]
```

In [49]:
# When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

# When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
# Assume Finished means the receipt is processed and accepted

query_total_items = """
SELECT 
    rewards_receipt_status AS status,
    SUM(purchased_item_count) AS total_items
FROM RECEIPT
WHERE rewards_receipt_status IN ('FINISHED', 'REJECTED')
GROUP BY rewards_receipt_status
ORDER BY total_items DESC;
"""

cursor.execute(query_total_items)
results = cursor.fetchall()

print("Rewards Receipt Status | Total Items Purchased")
print("-----------------------------------------------")
for status, total_items in results:
    print(f"{status} | {total_items}")



Rewards Receipt Status | Total Items Purchased
-----------------------------------------------
FINISHED | 8184
REJECTED | 173


In [ ]:
The total number of items purchased from `finished` receipts > the `rejected` receipts

```py
Rewards Receipt Status | Total Items Purchased
-----------------------------------------------
FINISHED | 8184
REJECTED | 173
```

In [53]:
# Which brand has the most spend among users who were created within the past 6 months?

query = """
WITH max_date AS (
    SELECT max(created_date) AS max_created_date
    FROM USER
),
recent_users AS (
    SELECT user_id
    FROM USER, max_date
    WHERE created_date >= date(max_created_date, '-6 months')
),
brand_spend AS (
    SELECT 
        b.brand_id,
        b.name,
        SUM(CAST(r.total_spent AS REAL)) AS total_spend
    FROM RECEIPT r
    JOIN RECEIPT_ITEM ri ON r.receipt_id = ri.receipt_id
    JOIN BRAND b ON ri.brand_code = b.brand_code
    WHERE r.user_id IN (SELECT user_id FROM recent_users)
    GROUP BY b.brand_id, b.name
)
SELECT *
FROM recent_users

"""

cursor.execute(query)
result = cursor.fetchone()

print("Brand with Most Spend Among Users Created Within the Past 6 Months:")
print("---------------------------------------------------------------------")
if result:
    print(f"Brand ID: {result[0]}")

Brand with Most Spend Among Users Created Within the Past 6 Months:
---------------------------------------------------------------------
